In [ ]:
pip install phe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


linear regression model

In [ ]:
import pandas as pd
import numpy as np
import phe as paillier
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

class LinModel:
	def __init__(self):
		pass

	def getResults(self):
		df=pd.read_csv('https://raw.githubusercontent.com/satssehgal/Homomorphic-Encryption/master/employee_data.csv')
		y=df.salary
		X=df.drop('salary',axis=1)
		X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
		reg = LinearRegression().fit(X_train, y_train)
		y_pred=reg.predict(X_test)
		RMSE=pow(mean_squared_error(y_pred, y_test),0.5)
		R=r2_score(y_pred, y_test)
		return reg, y_pred, RMSE, R

	def getCoef(self):
		return self.getResults()[0].coef_


def main():
	cof=LinModel().getCoef()
	print(cof)


if __name__=='__main__':
	main()






[-3.97674029e-01  4.69364136e+02 -1.85674425e+02  1.72099476e+01]


customer

In [ ]:
import phe as paillier
import json

def storeKeys():
	public_key, private_key = paillier.generate_paillier_keypair()
	keys={}
	keys['public_key'] = {'n': public_key.n}
	keys['private_key'] = {'p': private_key.p,'q':private_key.q}
	with open('custkeys.json', 'w') as file: 
		json.dump(keys, file)
storeKeys()  

def getKeys():
	with open('custkeys.json', 'r') as file: 
		keys=json.load(file)
		pub_key=paillier.PaillierPublicKey(n=int(keys['public_key']['n']))
		priv_key=paillier.PaillierPrivateKey(pub_key,keys['private_key']['p'],keys['private_key']['q'])
		return pub_key, priv_key 

def serializeData(public_key, data):
	encrypted_data_list = [public_key.encrypt(x) for x in data]
	encrypted_data={}
	encrypted_data['public_key'] = {'n': public_key.n}
	encrypted_data['values'] = [(str(x.ciphertext()), x.exponent) for x in encrypted_data_list]
	serialized = json.dumps(encrypted_data)
	return serialized



pub_key, priv_key = getKeys()
data = age, he, al, gen = [36,5,5,1]
serializeData(pub_key, data)
datafile=serializeData(pub_key, data)
with open('data.json', 'w') as file:
    json.dump(datafile, file)


	


server

In [ ]:
import phe as paillier
import json

def getData():
	with open('data.json', 'r') as file: 
		d=json.load(file)
	data=json.loads(d)
	return data

def computeData():
	data=getData()
	mycoef=LinModel().getCoef()
	pk=data['public_key']
	pubkey= paillier.PaillierPublicKey(n=int(pk['n']))
	enc_nums_rec = [paillier.EncryptedNumber(pubkey, int(x[0], int(x[1]))) for x in data['values']]
	results=sum([mycoef[i]*enc_nums_rec[i] for i in range(len(mycoef))])
	return results, pubkey

def serializeData():
	results, pubkey = computeData()
	encrypted_data={}
	encrypted_data['pubkey'] = {'n': pubkey.n}
	encrypted_data['values'] = (str(results.ciphertext()), results.exponent)
	serialized = json.dumps(encrypted_data)
	return serialized

def main():
	datafile=serializeData()
	with open('answer.json', 'w') as file:
		json.dump(datafile, file)

if __name__=='__main__':
	main()

loading of answer in customer

In [ ]:
def loadAnswer():
    with open('answer.json', 'r') as file: 
      ans=json.load(file)
      answer=json.loads(ans)
      return answer

answer_file=loadAnswer()
answer_key=paillier.PaillierPublicKey(n=int(answer_file['pubkey']['n']))
answer = paillier.EncryptedNumber(answer_key, int(answer_file['values'][0]), int(answer_file['values'][1]))
if (answer_key==pub_key):
    print(priv_key.decrypt(answer))

1421.3422358949524
